In [18]:
import zarr
from tifffile import TiffFile
from pathlib import Path
# Open the OME-TIFF file
path = '/data/etienne.doumazane/theremia/hest_data/wsis/INT1.tif'
path = 'data/INT1.tif'
path = Path(path)
dest_path = path.with_suffix('.zarr')
with TiffFile(path) as tif:
    # Create a Zarr group to store the pyramid
    zarr_group = zarr.open_group(dest_path, mode='w')

    # Loop through each resolution level in the pyramid
    for level, page in enumerate(tif.pages):
        data = page.asarray()  # Read the data for this resolution
        resolution_group = zarr_group.create_dataset(
            str(level),  # Name each resolution
            data=data,             # Write the data
            chunks=(256, 256),     # Define chunk size (adjust as needed)
            compressor=zarr.Blosc(cname='zstd', clevel=5)  # Compression settings
        )
        print(f"Written resolution {level} with shape {data.shape}")

print("Pyramidal data written to Zarr store.")


Written resolution 0 with shape (19968, 19200, 3)
Written resolution 1 with shape (9984, 9600, 3)
Written resolution 2 with shape (4992, 4800, 3)
Written resolution 3 with shape (2496, 2400, 3)
Written resolution 4 with shape (1248, 1200, 3)
Written resolution 5 with shape (624, 600, 3)
Written resolution 6 with shape (312, 300, 3)
Written resolution 7 with shape (156, 150, 3)
Pyramidal data written to Zarr store.


In [ ]:
import zarr
from tifffile import TiffFile

# Open the OME-TIFF file
with TiffFile('/data/etienne.doumazane/theremia/hest_data/wsis/INT1.tif') as tif:
    # Create a Zarr group to store the pyramid
    zarr_group = zarr.open_group('s3://renier-bucket-dev2024/etienne.doumazane/tmp/INT1.zarr', mode='w')

    # Loop through each resolution level in the pyramid
    for level, page in enumerate(tif.pages):
        data = page.asarray()  # Read the data for this resolution
        resolution_group = zarr_group.create_dataset(
            f'resolution_{level}',  # Name each resolution
            data=data,             # Write the data
            chunks=(256, 256),     # Define chunk size (adjust as needed)
            compressor=zarr.Blosc(cname='zstd', clevel=5)  # Compression settings
        )
        print(f"Written resolution {level} with shape {data.shape}")

print("Pyramidal data written to Zarr store.")


Written resolution 0 with shape (19968, 19200, 3)
Written resolution 1 with shape (9984, 9600, 3)
Written resolution 2 with shape (4992, 4800, 3)
Written resolution 3 with shape (2496, 2400, 3)
Written resolution 4 with shape (1248, 1200, 3)
Written resolution 5 with shape (624, 600, 3)
Written resolution 6 with shape (312, 300, 3)
Written resolution 7 with shape (156, 150, 3)
Pyramidal data written to Zarr store.


In [ ]:
# with TiffFile('/data/etienne.doumazane/theremia/hest_data/wsis/INT1.tif') as tif:
#     # Access the first series (image pyramid level or dataset)

#     # Loop through each page (frame) in the series
#     for page_index, page in enumerate(tif.pages):
#         if page.is_tiled:
#             print(f"Page {page_index}: Tiled")
#             print(f"Tile Width: {page.tilewidth}")
#             print(f"Tile Length: {page.tilelength}")
#         else:
#             print(f"Page {page_index}: Striped")
#             print(f"Rows Per Strip: {page.rowsperstrip}")


In [ ]:
import napari
import zarr
from tifffile import TiffFile
import dask.array as da

uri = "s3://renier-bucket-dev2024/etienne.doumazane/wsis/INT1.zarr/"
store = zarr.open(uri, mode="r")
multiscale_data = [da.from_zarr(store[str(i)]) for i in range(8)]


In [15]:
name = "NCBI439"
uri = f"s3://renier-bucket-dev2024/etienne.doumazane/wsis/{name}.zarr/"
store = zarr.open(uri, mode="r")
multiscale_data = [da.from_zarr(store[str(i)]) for i in range(7)]

In [9]:
v = napari.Viewer()

In [16]:
v.add_image(multiscale_data)

<Image layer 'multiscale_data [1]' at 0x3404a0f70>

In [13]:
v.layers["Shapes"].data


[array([[6310.65258726, 7233.05685294],
        [6278.98117511, 7114.28905738],
        [6207.72049777, 6932.17843752],
        [6081.03484917, 6409.60013705],
        [6065.1991431 , 6330.42160668],
        [6025.60987791, 6029.54319126],
        [6025.60987791, 5879.10398355],
        [6065.1991431 , 5728.66477584],
        [6128.5419674 , 5601.97912724],
        [6191.8847917 , 5499.04703775],
        [6271.06332207, 5419.86850738],
        [6358.15970548, 5348.60783004],
        [6469.009648  , 5285.26500574],
        [6627.36670875, 5237.75788752],
        [6754.05235735, 5253.59359359],
        [6999.50580151, 5340.689977  ],
        [7118.27359707, 5411.95065434],
        [7181.61642137, 5475.29347864],
        [7237.04139263, 5546.55415598],
        [7292.46636389, 5625.73268635],
        [7339.97348212, 5712.82906976],
        [7379.5627473 , 5847.4325714 ],
        [7395.39845338, 5926.61110177],
        [7419.15201249, 6077.05030948],
        [7427.06986553, 6338.33945972],


In [22]:
store

<zarr.hierarchy.Group '/'>

In [8]:
from st_challenge.utils import dump_yaml

In [9]:
from pathlib import Path

In [13]:
data = {"names": [path.stem for path in sorted(Path('/data/etienne.doumazane/theremia/hest_data/wsis/').glob("*.tif"))]}
dump_yaml(data, "wsi_names.yaml")